## MNIST challenge - ConvNet model : tensorflow 1.14


In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 

print(tf.__version__)

2.4.1


In [4]:
from tensorflow.python.framework import ops

def reset_graph(seed=42):
    ops.reset_defult_graph()
    tf.random.set_seed(seed)
    np.random.seed(seed)

build the CNN architecture

In [ ]:
reset_graph()

with tf.name_scope("input"):
    X = tf.placeholder(tf.float32, shape=[None, 28*28], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, 28, 28, 1])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=32, kernel_size=3, strides=1, padding="SAME", activation=tf.nn.relu, name="conv1")

conv2 = tf.layers.conv2d(X_reshaped, filters=64, kernel_size=3, strides=1, padding="SAME", activation=tf.nn.relu, name="conv1")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], stride=[1,2,2,1], padding="VAlID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_flat*7*7])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, 64, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, 10, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()


load dataset

In [ ]:
train_df = mnist_tr_df = pd.read_csv('../../data/datasets/train.csv', header=0).sample(frac=1)
y_train = train_df.iloc[:, 0].values
X_train = train_df.iloc[:, 1:].values
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



In [ ]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [ ]:
train the model on loaded dataset

In [ ]:
n_epochs = 10
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Last batch accuracy:", acc_batch, "Test accuracy:", acc_test)

        save_path = saver.save(sess, "./my_mnist_model")